In [2]:
import os
import sys
from google.colab import drive
import glob
from pathlib import Path
import shutil
from tqdm.auto import tqdm
def make_dirs():
    # make dirs for models & logs
    dirs = [
        "/content/models",

        "/content/models/segmentation",
        "/content/models/classification",
        "/content/models/autoencoder",

        "/content/models/segmentation/unet",
        "/content/models/segmentation/segformer",
        "/content/models/classification/efficienet",
        "/content/models/classification/convnext",

        "/content/models_log",

        "/content/models_log/segmentation",
        "/content/models_log/segmentation/unet",
        "/content/models_log/segmentation/segformer",

        "/content/models_log/classification",
        "/content/models_log/classification/efficienet",
        "/content/models_log/classification/convnext",
        "/content/models_log/autoencoder",
    ]
    for dir in dirs:
        os.makedirs(dir, exist_ok=True)



def mount_drive():

    drive.mount('/content/drive')
    source = '/content/drive/MyDrive/datasets/multi'  # Na Drive
    destination = '/content/datasets/multi'            # Link lokalny

    os.makedirs('/content/datasets', exist_ok=True)

    if os.path.exists(destination):
        if os.path.islink(destination):
            os.unlink(destination)
        else:
            shutil.rmtree(destination)

    # os.symlink(source, destination)
    print(f"Copying dataset from {source} to {destination}...")

    # Count files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(source)])

    with tqdm(total=total_files, desc="Copying", unit="file") as pbar:
        def copy_func(src, dst):
            shutil.copy2(src, dst)
            pbar.update(1)

        shutil.copytree(source, destination, copy_function=copy_func)

    print(f"Dataset copied!")
    print(f"  {destination} <- {source}")

def set_colab(path: str, on_colab: bool):
    dir = Path(path)
    result = list(dir.rglob("hparams.[pP][yY]"))
    for r in result:
        with open(r, "r+") as f:
            pos = f.tell()
            line = f.readline()
            if on_colab:
                if line == "ON_COLAB = False\n":
                    f.seek(pos)
                    f.write("ON_COLAB = True\n")
            else:
                if line == "ON_COLAB = True\n":
                    f.seek(pos)
                    f.write("ON_COLAB = False\n")

def set_hparams(unet_epochs=15, segformer_epochs=15):
    """
    Zmienia wartości hiperparametrów w pliku hparams.py

    Args:
        unet_epochs (int, optional): Liczba epok dla U-Net. Default: 15
        segformer_epochs (int, optional): Liczba epok dla SegFormer. Default: 15

    Returns:
        bool: True jeśli sukces, False jeśli błąd
    """
    import re

    hparams_file = "/home/krzeslaav/Projects/bachlor/src/segmentation/common/hparams.py"

    try:
        # Wczytaj zawartość pliku
        with open(hparams_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # Zmień UNET_EPOCHS
        content = re.sub(
            r'UNET_EPOCHS = \d+',
            f'UNET_EPOCHS = {unet_epochs}',
            content
        )

        # Zmień SEGFORMER_EPOCHS
        content = re.sub(
            r'SEGFORMER_EPOCHS = \d+',
            f'SEGFORMER_EPOCHS = {segformer_epochs}',
            content
        )

        # Zapisz zmieniony plik
        with open(hparams_file, 'w', encoding='utf-8') as f:
            f.write(content)

        print(f"✓ Ustawiono parametry:")
        print(f"  - UNET_EPOCHS = {unet_epochs}")
        print(f"  - SEGFORMER_EPOCHS = {segformer_epochs}")
        return True

    except Exception as e:
        print(f"✗ Błąd: {e}")
        return False
def set_resume_checkpoint(checkpoint_path=None):
    """
    Zmienia wartość RESUME_CHECKPOINT w pliku train.py

    Args:
        checkpoint_path (str, optional): Ścieżka do checkpointa.
                                        Jeśli None, ustawia RESUME_CHECKPOINT = None

    Returns:
        bool: True jeśli sukces, False jeśli błąd
    """
    import re

    train_file = "/home/krzeslaav/Projects/bachlor/src/segmentation/unet/train.py"

    try:
        # Wczytaj zawartość pliku
        with open(train_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # Przygotuj nową wartość
        if checkpoint_path is None:
            new_value = "RESUME_CHECKPOINT = None"
        else:
            # Escapuj backslashe dla Windows paths
            escaped_path = checkpoint_path.replace('\\', '\\\\')
            new_value = f'RESUME_CHECKPOINT = "{escaped_path}"'

        # Zamień linie z RESUME_CHECKPOINT
        pattern = r'RESUME_CHECKPOINT = .*'
        content = re.sub(pattern, new_value, content)

        # Zapisz zmieniony plik
        with open(train_file, 'w', encoding='utf-8') as f:
            f.write(content)

        print(f"✓ Ustawiono: {new_value}")
        return True

    except Exception as e:
        print(f"✗ Błąd: {e}")
        return False

In [ ]:
!sudo apt-get -qq install zstd
!gdown --id 1ZCY_ewakkoxohu4qqk4kHHgBCqi0urIn -O dataset_segmentation.tar.zst
!mkdir /content/datasets
!tar -xf /content/dataset_segmentation.tar.zst -C /content/datasets/
!mv /content/datasets/dataset_segmentation/ /content/datasets/multi/



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (o

In [ ]:
!pip install -q segmentation-models-pytorch torchmetrics
!pip install -q torchmetrics
!git clone -q https://github.com/mszczesniak02/bachlor_backup.git
make_dirs()
set_colab("/content/bachlor_backup/src", True)

!cat bachlor_backup/src/segmentation/common/hparams.py | head -1
!cat bachlor_backup/src/classification/common/hparams.py | head -1
!cat bachlor_backup/src/autoencoder/hparams.py | head -1
!cat bachlor_backup/src/dim_translator/hparams.py | head -1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 62.6 MB/s eta 0:00:00
ON_COLAB = True
ON_COLAB = True
ON_COLAB = True
cat: bachlor_backup/src/dim_translator/hparams.py: No such file or directory


In [ ]:
!rm -rd bachlor_backup/
!git clone -q https://github.com/mszczesniak02/bachlor_backup.git
make_dirs()
set_colab("/content/bachlor_backup/src", True)

!cat bachlor_backup/src/segmentation/common/hparams.py | head -1
!cat bachlor_backup/src/classification/common/hparams.py | head -1
!cat bachlor_backup/src/autoencoder/hparams.py | head -1
!cat bachlor_backup/src/dim_translator/hparams.py | head -1


ON_COLAB = True
ON_COLAB = True
ON_COLAB = True
cat: bachlor_backup/src/dim_translator/hparams.py: No such file or directory


In [ ]:
!cd bachlor_backup/src/segmentation/unet/ && python train.py

In [ ]:

original_sys_path = sys.path.copy()

# sys.path.append('/content/bachlor_backup/src/autoencoder')
# import train as autoencoder_train

sys.path.append('/content/bachlor_backup/src/segmentation/unet/')
import train as unet_train

# sys.path.append('/content/bachlor_backup/src/segmentation/segformer/')
# import train as segformer_train

# sys.path.append('/content/bachlor_backup/src/classification/convnext')
# import train as convnext_train

# sys.path.append('/content/bachlor_backup/src/classification/efficienet')
# import train as efficienet_train


In [ ]:
unet_train.main()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')